# Module 3 – Case Study Mini-Project

In this notebook I go through a full data preparation pipeline for a **retail store sales** dataset.
The goal is to clean and transform the data so that it is ready for a future machine learning model
that could predict the **total amount spent per transaction**.

## Part 0 – Setup and Data Loading

In [ ]:
# I import the main libraries I usually use for this kind of analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# I load the raw retail store sales data
# I also tell pandas that '?' should be treated as a missing value just in case
csv_file = 'retail_store_sales_3b2f9e32-25a6-4de3-a38a-efafad9ca724.csv'
df = pd.read_csv(csv_file, na_values='?')

# I quickly check the shape so I know how many rows and columns I am working with
df.shape

## Part 1 – Exploratory Data Analysis (EDA)

### 1.1 Initial Diagnosis

In [ ]:
# I want to see a basic overview of the dataframe structure
df.info()

In [ ]:
# I check the first few rows to understand what the columns look like
df.head()

In [ ]:
# I use describe() to get summary statistics for the numerical features
df.describe()

In [ ]:
# And I also look at a describe with include='object' to see the categorical side
df.describe(include='object')

### 1.2 Univariate Analysis

In [ ]:
# I pick a few important numerical columns and plot histograms
num_cols = ['Price Per Unit', 'Quantity', 'Total Spent']
plt.figure(figsize=(15, 4))
for i, col in enumerate(num_cols, 1):
    plt.subplot(1, 3, i)
    df[col].hist(bins=30)
    plt.title(col)
plt.tight_layout()
plt.show()

In [ ]:
# I do the same for some key categorical variables using bar plots
cat_cols = ['Category', 'Payment Method', 'Location']
plt.figure(figsize=(15, 4))
for i, col in enumerate(cat_cols, 1):
    plt.subplot(1, 3, i)
    df[col].value_counts().plot(kind='bar')
    plt.title(col)
plt.tight_layout()
plt.show()

### 1.3 Bivariate Analysis

In [ ]:
# I look at the relationship between Quantity and Total Spent with a scatter plot
plt.figure(figsize=(6, 4))
plt.scatter(df['Quantity'], df['Total Spent'], alpha=0.3)
plt.xlabel('Quantity')
plt.ylabel('Total Spent')
plt.title('Quantity vs Total Spent')
plt.show()

In [ ]:
# I also check the correlation between the numerical columns
numeric_df = df[['Price Per Unit', 'Quantity', 'Total Spent']]
corr = numeric_df.corr()

plt.figure(figsize=(4, 3))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='viridis')
plt.title('Correlation Heatmap (Numerical Features)')
plt.show()

### 1.4 EDA Summary

From the first exploration I notice the following points:

1. There are **missing values** in several columns such as `Item`, `Price Per Unit`, `Quantity`, `Total Spent` and especially `Discount Applied`.
2. The numerical variables (especially `Total Spent`) are **right-skewed**, which is expected because most transactions are small and a few are very large.
3. `Quantity` and `Total Spent` show a clear **positive relationship**, as larger quantities usually lead to larger total amounts.
4. Some payment methods and locations are clearly more frequent than others, which might influence the total spend.
5. The dataset already has a reasonable set of features, but it will still benefit from some extra feature engineering around dates and locations later.

## Part 2 – Data Cleaning

### 2.1 Missing Values

In [ ]:
# I check how many missing values I have in each column
df.isna().sum()

For the missing values I use the following strategy:

- For **`Item`** (categorical), I fill missing values with the **most frequent item** (mode).
- For **`Discount Applied`** (categorical), I treat missing values as a separate category `'Unknown'`.
- For the numerical trio **`Price Per Unit`**, **`Quantity`**, and **`Total Spent`**, I first try to
  **reconstruct the missing value using the other two** (because `Total Spent = Price Per Unit * Quantity`).
  If I still have missing values after that, I fill them with the **median**, because the distributions
  are skewed and I do not want extreme values to influence the imputation too much.

In [ ]:
# I copy the dataframe so I keep the original version intact
df_clean = df.copy()

# 1) Fill missing Item with the most frequent value
most_frequent_item = df_clean['Item'].mode()[0]
df_clean['Item'] = df_clean['Item'].fillna(most_frequent_item)

# 2) Treat missing Discount Applied as 'Unknown'
df_clean['Discount Applied'] = df_clean['Discount Applied'].fillna('Unknown')

# 3) Reconstruct missing numerical values if possible
# If Price Per Unit is missing but Quantity and Total Spent are present
mask_price_missing = df_clean['Price Per Unit'].isna() & df_clean['Quantity'].notna() & df_clean['Total Spent'].notna()
df_clean.loc[mask_price_missing, 'Price Per Unit'] = (
    df_clean.loc[mask_price_missing, 'Total Spent'] / df_clean.loc[mask_price_missing, 'Quantity']
)

# If Quantity is missing but Price Per Unit and Total Spent are present
mask_quantity_missing = df_clean['Quantity'].isna() & df_clean['Price Per Unit'].notna() & df_clean['Total Spent'].notna()
df_clean.loc[mask_quantity_missing, 'Quantity'] = (
    df_clean.loc[mask_quantity_missing, 'Total Spent'] / df_clean.loc[mask_quantity_missing, 'Price Per Unit']
)

# If Total Spent is missing but Price Per Unit and Quantity are present
mask_total_missing = df_clean['Total Spent'].isna() & df_clean['Price Per Unit'].notna() & df_clean['Quantity'].notna()
df_clean.loc[mask_total_missing, 'Total Spent'] = (
    df_clean.loc[mask_total_missing, 'Price Per Unit'] * df_clean.loc[mask_total_missing, 'Quantity']
)

# After reconstruction, I still fill any remaining numeric missing values with the median
for col in ['Price Per Unit', 'Quantity', 'Total Spent']:
    median_value = df_clean[col].median()
    df_clean[col] = df_clean[col].fillna(median_value)

# I double-check that I do not have missing values left in these key columns
df_clean[['Item', 'Price Per Unit', 'Quantity', 'Total Spent', 'Discount Applied']].isna().sum()

### 2.2 Data Type Correction

In [ ]:
# I convert Transaction Date to a proper datetime type
df_clean['Transaction Date'] = pd.to_datetime(df_clean['Transaction Date'], errors='coerce')

# I can also treat some object columns as categories
categorical_columns = ['Transaction ID', 'Customer ID', 'Category', 'Item',
                       'Payment Method', 'Location', 'Discount Applied']
for col in categorical_columns:
    df_clean[col] = df_clean[col].astype('category')

df_clean.dtypes

### 2.3 Outliers (Optional)

In [ ]:
# I quickly check for outliers in Total Spent using a boxplot
plt.figure(figsize=(4, 4))
plt.boxplot(df_clean['Total Spent'])
plt.title('Boxplot of Total Spent')
plt.ylabel('Total Spent')
plt.show()

In [ ]:
# I implement a simple IQR-based capping to reduce the impact of extreme outliers
Q1 = df_clean['Total Spent'].quantile(0.25)
Q3 = df_clean['Total Spent'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# I cap the values outside the bounds
df_clean['Total Spent'] = np.where(
    df_clean['Total Spent'] < lower_bound, lower_bound,
    np.where(df_clean['Total Spent'] > upper_bound, upper_bound, df_clean['Total Spent'])
)

# I check the boxplot again after capping
plt.figure(figsize=(4, 4))
plt.boxplot(df_clean['Total Spent'])
plt.title('Boxplot of Total Spent (After Capping)')
plt.ylabel('Total Spent')
plt.show()

## Part 3 – Feature Engineering

I create at least two new features that might be useful for a future model:

1. **`Transaction_Month`** – the month of the transaction (1–12). This can capture seasonality or monthly patterns.
2. **`Is_Online`** – a binary feature that is 1 if the location is `'Online'` and 0 otherwise. This can capture
   the difference between online and offline sales.

In [ ]:
# I create the new features based on the cleaned dataframe
df_fe = df_clean.copy()

# Month of the transaction (I fill NaT just in case there were parsing issues)
df_fe['Transaction_Month'] = df_fe['Transaction Date'].dt.month.fillna(df_fe['Transaction Date'].dt.month.mode()[0])

# Binary feature: 1 if Online, 0 otherwise
df_fe['Is_Online'] = (df_fe['Location'] == 'Online').astype(int)

df_fe[['Transaction Date', 'Transaction_Month', 'Location', 'Is_Online']].head()

## Part 4 – Data Transformation (Model-Ready Dataset)

For the model-ready dataset I:

- Choose **`Total Spent`** as the target variable (this represents how much the customer paid).
- Keep **numerical features** like `Price Per Unit`, `Quantity` and the engineered `Transaction_Month`.
- Keep **categorical features** such as `Category`, `Item`, `Payment Method`, `Location`, and `Discount Applied`,
  plus the binary `Is_Online` feature.
- Apply **one-hot encoding** to the categorical features.
- Use **StandardScaler** to scale the main numerical features.

In [ ]:
from sklearn.preprocessing import StandardScaler

# I define my target and the features I want to keep
target_col = 'Total Spent'

numeric_features = ['Price Per Unit', 'Quantity', 'Transaction_Month']
categorical_features = ['Category', 'Item', 'Payment Method', 'Location', 'Discount Applied']
binary_features = ['Is_Online']

selected_columns = numeric_features + categorical_features + binary_features + [target_col]
model_df = df_fe[selected_columns].copy()

# I separate features and target
X = model_df.drop(columns=[target_col])
y = model_df[target_col]

# I apply one-hot encoding to all categorical columns
X_encoded = pd.get_dummies(X, columns=categorical_features, drop_first=True)

# I scale only the main continuous numeric features
scaler = StandardScaler()
X_encoded[numeric_features] = scaler.fit_transform(X_encoded[numeric_features])

X_encoded.head()

### 4.1 Final Model-Ready Dataset

In [ ]:
# I combine the final feature matrix and the target into a single dataframe for inspection
final_df = X_encoded.copy()
final_df[target_col] = y

# I check the first few rows of the fully numerical dataset
final_df.head()

## Final Notes

- The dataset is now cleaned, all important missing values are handled.
- Outliers in `Total Spent` are capped to reduce their influence.
- I created useful new features related to transaction month and online vs offline sales.
- All categorical columns are encoded and the main numerical columns are scaled.

This notebook can now be used as a base for building and testing machine learning models
for predicting the total amount spent per transaction.